In [ ]:
# Fits a hyperbolic curve on a given data.
# Author: @jasminebaclig

# Sets base path on Google Drive for CSV data.
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/'

# Imports other necessary packages and functions.
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np

In [ ]:
'''
TODO: Change the path and/or the file name to match the location and name of your CSV file.
  - If your CSV file is in a folder in your Google Drive, add "folder-name/" in front of the CSV file name.
      Example: "folder/file.csv" or "folder/subfolder/file.csv" (include the quotation marks)
  - The variable base_path directs the read_csv function to look for the CSV file in your Google Drive.
    If the file is only shared with you, you need to make a copy or a shortcut of it in your Drive.
'''
# Reads the CSV file with the data into a data frame (table).
data = pd.read_csv(base_path + "file_name.csv")


'''
TODO: Change the column labels inside data[] to match the column labels for the dependent and independent
      variables in your CSV file.
'''
# Extracts the x and y data from the data frame.
x_data = data['x_values']
y_data = data['y_values']


'''
TODO: Change the labels for the axes of the graph.
'''
# Assigns the axes labels to variables.
x_label = "Independent Variable (units)"
y_label = "Dependent Variable (units)"

# Graphs a scatter plot of the data points.
plt.scatter(x_data, y_data)
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.show()

In [ ]:
# Defines the function for a hyperbolic curve with the form y = (b * x) / (a + x).
def hyperbolic_func(x, b, a):
  return (b * x) / (a + x)


'''
TODO: Change the estimated parameters stored in the variable param_guess based on the scatter plot
      of your data generated above. You do not have to be precise with your guesses; just a rough estimate is okay.
  - First parameter (b): y value of the horizontal asymptote.
  - Second parameter (a): x value at half of the asymptote's y-value.
      Example: [5, 20] assigns an initial guess of 5 to b and 20 to a
'''
# Finds the best parameters for the hyperbolic function using the initial guesses in param_guess.
# Returns the optimized parameters and their estimated covariance.
param_guess = [-100, -100]
param_optimized, param_covar = curve_fit(hyperbolic_func, x_data, y_data, param_guess)
b_optimized = param_optimized[0]
a_optimized = param_optimized[1]


'''
TODO: Change the arguments of np.linspace() to generate an appropriate list of x values with a range that matches
      the range of your independent variable. This will determine how the graph of the fitted function will look.
  - First argument: starting value of the range (x-value where you want the graph to start)
  - Second argument: ending value of the range (x-value where you want the graph to end)
  - Third argument: number of evenly distributed values within the specified range (higher value -> smoother graph)
      Example: linspace(0, 500, 100) generates a list of 100 equally spaced values starting from 0 and ending in 500
'''
# Generates predicted y values of the hyperbolic function using the specified range of x values.
# Allows the fitted function to be graphed separately from the experimental data.
x_pred = np.linspace(0, 1, 1)
y_pred = hyperbolic_func(x_pred, b_optimized, a_optimized)

# Graphs the data with the fitted hyperbolic curve.
plt.scatter(x_data, y_data, label = "Collected Data")
plt.plot(x_pred, y_pred, color = "black", label = "Fitted Function")
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.legend()
plt.show()

In [ ]:
# Determines the standard deviation errors of the optimized parameters.
param_std_error = np.sqrt(np.diag(param_covar))

# Prints the optimized parameters and the associated standard deviation errors.
print(f"The optimized value for b is {b_optimized}.")
print(f"The standard deviation error for b is {param_std_error[0]}.\n")
print(f"The optimized value for a is {a_optimized}.")
print(f"The standard deviation error for a is {param_std_error[1]}.\n")

# Determines the total sum of squares of the data.
total_sum_sq = np.sum((y_data - y_data.mean())**2)

# Determines the residual sum of squares of the data and its fit.
y_pred_data = hyperbolic_func(x_data, b_optimized, a_optimized)
resid_sum_sq = np.sum((y_data - y_pred_data)**2)

# Calculates and prints the R2 value describing the fit of the curve on the data.
r2 = 1 - (resid_sum_sq / total_sum_sq)
print(f"The R2 coefficient of the hyperbolic fit is {r2:.4f}.")